# OTROS TIEMPOS

In [60]:
weeks = 56

In [61]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2024-10-02 hasta 2025-10-29


In [62]:
df.head(2)

,create_uid,__last_update,wk_line_id,create_date,name,tipo,tarea_id,delay_pausa,is_pause,finalizo,delay_parcial,delay_total,production_id,inicio,write_date,workcenter_id,costo_total,write_uid,id,delay_hours,display_name,tarea
0,"[345, Planta1]",2024-10-02 12:25:02,False,2024-10-02 11:50:02,"[609, MARIA PAULA PARRA PINILLA]",produccion,"[3, Recuperar cajas]",0.0,False,2024-10-02 07:25:02,0.0,0.0,False,2024-10-02 06:50:02,2024-10-02 12:25:02,False,7861.53,"[345, Planta1]",112,0.58,MARIA PAULA PARRA PINILLA,Recuperar cajas
1,"[345, Planta1]",2024-10-03 12:39:27,"[61916, EMBALAJE - DA2768 FILTRO AIRE KENWORTH, PETERBILT.]",2024-10-02 11:50:50,"[650, CLARA MARLEN OSPINA CASTRO]",sistema,"[2, Produccion]",89314.0,False,2024-10-03 07:39:27,13287.0,13287.0,"[11384, MO11336]",2024-10-02 06:50:51,2024-10-03 12:39:27,"[57, EMBALAJE]",7.38,"[345, Planta1]",113,3.69,CLARA MARLEN OSPINA CASTRO,Produccion


In [63]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

df_ot.columns = df_ot.columns.str.lower()
df_ot = df_ot.rename(columns={
    "display_name": "operario",
    "inicio": "fecha_inicio",
    "tipo": "actividad",
})

# Convertir fechas
for col in ["fecha_inicio", "create_date", "__last_update"]:
    df_ot[col] = pd.to_datetime(df_ot[col], errors="coerce")

# Agregar columnas que no existen en df_ot para igualar estructura
df_ot["duracion_horas"] = df_ot["delay_hours"]
df_ot["centro_produccion"] = "OTRAS TAREAS"

df_ot = df_ot[["operario","__last_update","actividad","finalizo","fecha_inicio","tarea","duracion_horas"]]

# Hora actual (en tu zona horaria si quieres)
ahora = pd.Timestamp.now()

df_ot["fecha_inicio"] = pd.to_datetime(df_ot["fecha_inicio"], errors="coerce")
df_ot["finalizo"] = pd.to_datetime(df_ot["finalizo"], errors="coerce")

# --- 1️⃣ Crear columna en_curso ---
# True si no hay fecha de finalización
df_ot["en_curso"] = df_ot["finalizo"].isna() | (df_ot["finalizo"] == False)

# --- 2️⃣ Recalcular duración ---
df_ot["duracion_horas"] = df_ot.apply(
    lambda row: (
        (ahora - row["fecha_inicio"]).total_seconds() / 3600
        if row["en_curso"] and pd.notna(row["fecha_inicio"])
        else (row["finalizo"] - row["fecha_inicio"]).total_seconds() / 3600
        if pd.notna(row["finalizo"]) and pd.notna(row["fecha_inicio"])
        else 0
    ),
    axis=1
).round(2)

In [64]:
df_ot.tail(5)

,operario,__last_update,actividad,finalizo,fecha_inicio,tarea,duracion_horas,en_curso
34788,JAVIER ESTEBAN GONZALEZ ARENAS,2025-10-29 18:55:41,produccion,2025-10-29 13:55:41,2025-10-29 12:36:36,Reproceso,1.32,False
34804,DAVID ESTEBAN MALAGON CETINA,2025-10-29 19:16:40,produccion,2025-10-29 14:16:40,2025-10-29 13:20:13,Alistamiento MP,0.94,False
34819,KATHERIN ANDREA GIL AGUILAR,2025-10-29 19:58:41,produccion,NaT,2025-10-29 14:58:41,Auxiliar producción,0.84,True
34827,DAVID ESTEBAN MALAGON CETINA,2025-10-29 20:26:33,produccion,NaT,2025-10-29 15:26:33,Alistamiento MP,0.37,True
34829,FRANCY CAROLINA PINZON CHACON,2025-10-29 20:36:15,produccion,NaT,2025-10-29 15:36:15,Orden y Aseo,0.21,True


# TIEMPOS POR OP

In [65]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd
import re

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op["nom_filtro"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\(([^)]+)\)", x).group(1) if re.search(r"\(([^)]+)\)", x) else None
)

df_op["product_ref"] = df_op["product_fabricado_id"].astype(str).apply(
    lambda x: re.search(r"\[([A-Z0-9]+)\]", x).group(1) if re.search(r"\[([A-Z0-9]+)\]", x) else None
)

df_op["maquina"] = df_op["fuerza_lab_id"].astype(str).apply(
    lambda x: re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE).group(1).strip()
    if re.search(r",\s*[\'\"]?([A-ZÁÉÍÓÚÑ0-9\s]+?)(?=\s*-\s*)", x, re.IGNORECASE)
    else None
)



df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id","product_fabricado_id"])

#Tarda aprox 4 minutos


📅 Descargando registros desde 2024-10-02 hasta 2025-10-29


In [66]:
# --- Limpieza df_op ---
df_op.columns = df_op.columns.str.lower()
df_op = df_op.rename(columns={"evento": "actividad", "display_name": "operario"})

# Asegurar tipos
for col in ["fecha_evento", "create_date", "write_date", "__last_update"]:
    df_op[col] = pd.to_datetime(df_op[col], errors="coerce")


In [67]:
from datetime import timedelta

# Asegurar tipos de fecha correctos
df_op["fecha_evento"] = pd.to_datetime(df_op["fecha_evento"], errors="coerce")

# --- 1️⃣ Crear columna 'final' ---
df_op["final"] = df_op.apply(
    lambda row: row["fecha_evento"] + timedelta(seconds=row["segundos_evento"])
    if pd.notna(row["segundos_evento"]) else row["fecha_evento"],
    axis=1
)

# --- 2️⃣ Ordenar por operario y fecha_evento ---
df_op = df_op.sort_values(["operario", "fecha_evento"])

# --- 3️⃣ Marcar el último evento por operario ---
df_op["ultimo_evento"] = df_op.groupby("operario")["fecha_evento"].transform("max")

# --- 4️⃣ Crear columna en_curso ---
# En curso = el evento más reciente del operario y su actividad es "iniciar"
df_op["en_curso"] = (df_op["fecha_evento"] == df_op["ultimo_evento"]) & (df_op["actividad"] == "iniciar")

# --- 5️⃣ Filtrar: eliminar 'iniciar' antiguos (solo mantener el último 'iniciar' si está en curso) ---
df_op_filtrado = df_op[
    (df_op["actividad"] != "iniciar") |  # mantener todo lo que no sea iniciar
    (df_op["en_curso"])                  # o mantener el último iniciar de cada operario
].copy()

# --- 6️⃣ Limpieza final ---
df_op_filtrado = df_op_filtrado.drop(columns=["ultimo_evento"])


In [68]:
#df_op_filtrado[df_op_filtrado["en_curso"]==True]
df_op_filtrado[(df_op_filtrado["operario"]=="JAMER PUPO SIERRA")& (df_op_filtrado["fecha_evento"]>"2025-10-21")].head(2)

,__last_update,horas_evento,create_date,fecha_evento,fuerza_lab_id,actividad,segundos_evento,write_date,operario,op,centro_produccion,nom_filtro,product_ref,maquina,final,en_curso
101898,2025-10-21 12:10:37,0.55,2025-10-21 12:10:37,2025-10-21 07:10:37,"[113786, ENTUBADORA 3 - DA8145 FILTRO AIRE EXCAVADORA RHINO REX210]",finalizar,1997.0,2025-10-21 12:10:37,JAMER PUPO SIERRA,MO13349,CORTE,025 DA8145,DAB08145025,ENTUBADORA 3,2025-10-21 07:43:54,False
101939,2025-10-21 12:43:35,0.52,2025-10-21 12:43:35,2025-10-21 07:43:35,"[113804, ENTUBADORA 3 - DA8169 FILTRO AIRE COSECHADORAS WORL]",finalizar,1864.0,2025-10-21 12:43:35,JAMER PUPO SIERRA,MO13352,CORTE,025 DA8169,DAB08169025,ENTUBADORA 3,2025-10-21 08:14:39,False


## UNIR DATAFRAMES

In [69]:
#df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/control_operarios.xlsx")

In [70]:
df_op_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52505 entries, 77 to 104825
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   __last_update      52505 non-null  datetime64[ns]
 1   horas_evento       52505 non-null  float64       
 2   create_date        52505 non-null  datetime64[ns]
 3   fecha_evento       52505 non-null  datetime64[ns]
 4   fuerza_lab_id      52505 non-null  object        
 5   actividad          52505 non-null  object        
 6   segundos_evento    52505 non-null  float64       
 7   write_date         52505 non-null  datetime64[ns]
 8   operario           52505 non-null  object        
 9   op                 52505 non-null  object        
 10  centro_produccion  52505 non-null  object        
 11  nom_filtro         52503 non-null  object        
 12  product_ref        52407 non-null  object        
 13  maquina            44058 non-null  object        
 14  final    

In [71]:
df_ot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6140 entries, 0 to 34829
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   operario        6140 non-null   object        
 1   __last_update   6140 non-null   datetime64[ns]
 2   actividad       6140 non-null   object        
 3   finalizo        6135 non-null   datetime64[ns]
 4   fecha_inicio    6140 non-null   datetime64[ns]
 5   tarea           6140 non-null   object        
 6   duracion_horas  6140 non-null   float64       
 7   en_curso        6140 non-null   bool          
dtypes: bool(1), datetime64[ns](3), float64(1), object(3)
memory usage: 389.7+ KB


In [72]:
# Copias de trabajo
df1 = df_op_filtrado.copy()
df2 = df_ot.copy()

# --- Homologar df_op_filtrado a estructura de df_ot ---
df1 = df1.rename(columns={
    'fecha_evento': 'fecha_inicio',
    'final': 'finalizo',
    'horas_evento': 'duracion_horas',
    'centro_produccion': 'tarea'
})

# --- Agregar columna de origen ---
df1['origen'] = 'produccion'
df2['origen'] = 'otras_tareas'

# --- Crear columna de segundos_evento ---
# En producción ya existe, pero renombramos para coherencia
if 'segundos_evento' not in df1.columns:
    df1['segundos_evento'] = df1['duracion_horas'] * 3600
# En tareas la calculamos desde duracion_horas
df2['segundos_evento'] = df2['duracion_horas'] * 3600

# --- Unir ambos sin perder datos ---
df_union = pd.concat([df1, df2], ignore_index=True, sort=False)

# --- Ordenar columnas ---
cols_base = [
    'operario', '__last_update', 'actividad', 'fecha_inicio', 'finalizo',
    'tarea', 'duracion_horas', 'segundos_evento', 'en_curso', 'origen'
]

cols_extra = [
    'op', 'fuerza_lab_id', 'maquina', 'nom_filtro', 'product_ref',
    'create_date', 'write_date'
]

cols_final = cols_base + [c for c in cols_extra if c in df_union.columns]
df_union = df_union[cols_final]


if 'tarea' in df_union.columns:
    df_union['tarea'] = df_union['tarea'].astype(str).str.upper()


# Tomar fecha y hora actual
ahora = pd.Timestamp.now()

# Filtrar filas en curso
mask_en_curso = df_union['en_curso'] == True

# Calcular diferencia de tiempo
df_union.loc[mask_en_curso, 'duracion_horas'] = (
    (ahora - df_union.loc[mask_en_curso, 'fecha_inicio']).dt.total_seconds() / 3600
)

df_union.loc[mask_en_curso, 'segundos_evento'] = (
    df_union.loc[mask_en_curso, 'duracion_horas'] * 3600
)

# Asegurar que la columna sea texto y eliminar espacios
df_union['op'] = df_union['op'].astype(str).str.strip()

df_union['operario'] = df_union['operario'].astype(str).str.strip()

df_union.loc[df_union['actividad'].str.lower().str.strip() == 'produccion', 'actividad'] = 'otras_tareas'

# Guardar a Excel
df_union.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/operarios_tiempo_completo.xlsx", index=False)

#TARDA 1 MINUTO


In [73]:
df_union[df_union["en_curso"]==True].head()

,operario,__last_update,actividad,fecha_inicio,finalizo,tarea,duracion_horas,segundos_evento,en_curso,origen,op,fuerza_lab_id,maquina,nom_filtro,product_ref,create_date,write_date
2448,ALVARO MELO PRADA,2025-10-29 20:44:15,iniciar,2025-10-29 15:44:15,2025-10-29 15:44:15,CORTE,0.132680,477.648958,True,produccion,MO13409,"[114615, DESPUNTE Y ENROLLADO - DA4738 FILTRO AIRE 2º KUBOTA, J. DEERE, BOBCAT, ALLMAN]",DESPUNTE Y ENROLLADO,025 DA4738,DAB04738025,2025-10-29 20:44:15,2025-10-29 20:44:15
4011,BLANCA CECILIA AVELLA ARIAS,2025-10-29 19:45:34,iniciar,2025-10-29 14:45:34,2025-10-29 14:45:34,INYECCION,1.110736,3998.648958,True,produccion,MO13393,"[114600, OMS - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",OMS,025 DA2570,DAB02570025,2025-10-29 19:45:34,2025-10-29 19:45:34
5892,BLANCA DORA CUBIDES SANCHEZ,2025-10-29 20:42:30,iniciar,2025-10-29 15:42:30,2025-10-29 15:42:30,CORTE,0.161847,582.648958,True,produccion,MO13407,"[114557, ENTUBADORA 3 - DA2690 FILTRO AIRE- FREIGTHLINER]",ENTUBADORA 3,025 DA2690,DAB02690025,2025-10-29 20:42:30,2025-10-29 20:42:30
10259,CLARA MARLEN OSPINA CASTRO,2025-10-29 20:46:46,iniciar,2025-10-29 15:46:46,2025-10-29 15:46:46,EMBALAJE,0.090736,326.648958,True,produccion,MO13313,"[114593, MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA9001 FILTRO AIRE 2° YOUTONG, LIUGONG]",None,025 DA9001,DAE09001025,2025-10-29 20:46:46,2025-10-29 20:46:46
11482,DANILO PUPO SIERRA,2025-10-29 15:15:04,iniciar,2025-10-29 10:15:04,2025-10-29 10:15:04,PLISADO,5.619069,20228.648958,True,produccion,MO13400,"[114544, PLISADORA 4 - DA2926 FILTRO AIRE DONSSON - INTERNATIONAL 4300 - 7400]",PLISADORA 4,025 DA2926,DAB02926025,2025-10-29 15:15:04,2025-10-29 15:15:04


In [74]:
#df_union[(df_union["operario"]=="JAMER PUPO SIERRA")& (df_union["fecha_inicio"]>"2025-10-21")].head(7)